
Homework2-Discovery of Frequent Itemsets and Association Rules 
By group 29:Disen Ling,Zhenghong Xiao

In this assignment we need to solve the following problems:

1. Finding frequent itemsets with support at least s;
2. Generating association rules with confidence at least c from the itemsets found in the first step.




Exercise 1

You are to solve the first sub-problem: to implement the A-Priori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions. Remind that support of an itemset is the number of transactions containing the itemset. To test and evaluate your implementation, write a program that uses your A-Priori algorithm implementation to discover frequent itemsets with support at least s in a given dataset of sales transactions which includes generated transactions (baskets) of hashed items


In [7]:
import itertools
# The itertools library is used to combine items to form candidate item sets

In [8]:
class A_priori():
    def __init__(self, threshold = 0.01, k = 3):
        self.threshold = threshold # at least 1% of frequency --- frequent
        self.k = k # the max size of the item set ([1,2,3]-3)
        self.min_nr_transactions = None # number of transactions to define frequent

    def read_dataset(self):
        transactions_dic = {} # build a dict to store key-value pairs (item-transaction)
        nr_transactions = 0 # number of transations which will be used in calculate support
        # read the dataset and store the data in transactions_dic 
        with open("data/T10I4D100K.dat", 'r') as file:
            for i, line in enumerate(file):
                nr_transactions += 1
                # get the items in one transaction
                transaction = [int(num) for num in line.strip().split(' ')]
                for item in transaction:
                # check if the item is already in the key-value pairs 
                    if item in transactions_dic.keys():
                        transactions_dic[item].append(i)
                    else:
                        transactions_dic[item] = [i]
        self.min_nr_transactions = nr_transactions * self.threshold
        # the minimum number of transactions a frequent item should have
        print("The transaction times threshold: ", self.min_nr_transactions)
        return transactions_dic
    
    # get the original frequent items like {[1] [2] [3]}
    def get_original_frequent_items(self, transactions_dic):
        original_frequent_items = {}
        for item in transactions_dic.keys():
            # check if the frequence is above the minimum frequence 1%
            if len(transactions_dic[item]) >= self.min_nr_transactions:
                # if so, add this item to the original frequent items
                original_frequent_items[item] = transactions_dic[item]
        print("Amount of frequent items: ", len(original_frequent_items))
        return original_frequent_items

    # use the current frequent item sets to generate the candidate frequent item sets for next filter
    def generate_candidates(self, original_frequent_items, frequent_items, factor):
        # the first filter from original frequent items to frequent item sets which have 2 items
        if factor == 2:
            candidates = list(itertools.combinations(original_frequent_items.keys(), factor))
        else:
            # get all the items in the frequent item sets and combine them based on the combination factor
            # [1,2] [2,3] [3,4] - 1,2,3,4 - [1,2,3] [2,3,4]
            candidates = set()
            for items in list(frequent_items.keys()):
                for item in items:
                    candidates.add(item)
            candidates = list(itertools.combinations(candidates, factor))
        return candidates
    
    # check if the candidates lists we generate are frequent
    def check_candidates(self, candidates, original_frequent_items):
        result = {} # the dic which is same to the original frequent items
        # pair - transactions
        for candidate in candidates:
                for i in range(0,len(candidate)):
                    if i == 0:
                        # initialize the interasection set
                        Interasection = set(original_frequent_items[candidate[i]])
                    # the interasection set is the set shows which transactions contains the candidate pait
                    Interasection = set.intersection(set(original_frequent_items[candidate[i]]),Interasection)
                if len(Interasection) >= self.min_nr_transactions:
                # check if this candidate is frequent
                    result[candidate] = Interasection
                    # the result is a map which contains frequent sets -- transactions
        return result


Exercise 2

Optional task for extra bonus: Solve the second sub-problem, i.e., develop and implement an algorithm for generating association rules between frequent itemsets discovered by using the A-Priori algorithm in a dataset of sales transactions. The rules must have support at least s and confidence at least c, where s and c are given as input parameters.


In [5]:
class Assosiate_rule():
    def __init__(self, confidence = 0.5):
        self.confidence = confidence # the confidence of assosiate-rule

    def assosiate(self, frequent_items, original_frequent_items):
        associations = [] # the association implements are stored in this list
        combinations = [] # combinations for frequent item pairs ([1,2,3] - [1,2] [1,3][2,3])
        items = frequent_items.keys() # item paris 
        for item in items:
            combinations.append(list(itertools.combinations(item, len(item) - 1)))
        
        # generate the X set and Y set which have the implement X --> Y
        for idx, item in enumerate(items):
            for combination in combinations[idx]:
                # the assosiate item: [2,3] -> [1,2,3] so [1] is the assosiate item
                assosiate_item = set(item) - set(combination)
                for i in range(0,len(item)):
                    if i == 0:
                        Interasection1 = set(original_frequent_items[item[i]])
                    Interasection1 = set.intersection(set(original_frequent_items[item[i]]),Interasection1)
                Numerator = len(Interasection1)
                # the frequence of the Y set (Support)
                for i in range(0,len(combination)):
                    if i == 0:
                        Interasection2 = set(original_frequent_items[combination[i]])
                    Interasection2 = set.intersection(set(original_frequent_items[combination[i]]),Interasection2)
                Denominator = len(Interasection2)
                # the frequence of the X set
                confidence = Numerator/Denominator # calculate the confidence
                if confidence >= self.confidence:
                    associations.append(str(combination)+"---→"+str(assosiate_item)+" : "+str(confidence))
        return associations

In [9]:
k = 3 # Maximum size of association k-itemset
a_priori = A_priori()
assosiate_rule = Assosiate_rule()
# get the dataset dict with key-value pairs (item-transactions)
transactions_dic = a_priori.read_dataset()
# get the first filter
original_frequent_items = a_priori.get_original_frequent_items(transactions_dic)
frequent_items = original_frequent_items

# 2nd to kth filter
for factor in range(2,k+1):
    print("Filter stage: ", factor)
    # get the frequent item set candidates
    candidates = a_priori.generate_candidates(original_frequent_items, frequent_items, factor)
    # check all the candidates and generate the result dict
    frequent_items = a_priori.check_candidates(candidates, original_frequent_items)
    # check the assosiation implements based on the frequent item sets
    assosiations = assosiate_rule.assosiate(frequent_items, original_frequent_items)
    # print the assosiation implements
    print(*assosiations,sep = "\n")
    for key in frequent_items.keys():
        print("Frequent pairs: ", key, "Times: ", len(frequent_items[key]))


The transaction times threshold:  1000.0
Amount of frequent items:  375
Filter stage:  2
(704,)---→{825} : 0.6142697881828316
(704,)---→{39} : 0.617056856187291
(227,)---→{390} : 0.577007700770077
Frequent pairs:  (368, 682) Times:  1193
Frequent pairs:  (368, 829) Times:  1194
Frequent pairs:  (825, 39) Times:  1187
Frequent pairs:  (825, 704) Times:  1102
Frequent pairs:  (39, 704) Times:  1107
Frequent pairs:  (227, 390) Times:  1049
Frequent pairs:  (390, 722) Times:  1042
Frequent pairs:  (217, 346) Times:  1336
Frequent pairs:  (789, 829) Times:  1194
Filter stage:  3
(704, 39)---→{825} : 0.9349593495934959
(704, 825)---→{39} : 0.9392014519056261
(39, 825)---→{704} : 0.8719460825610783
Frequent pairs:  (704, 39, 825) Times:  1035
